In [1]:
import os
from pathlib import Path
from torch.utils.data import DataLoader
import webdataset as wds

In [3]:
hf_token = os.environ['CMERAKI_HF_TOKEN']
url = "https://huggingface.co/datasets/cmeraki/tts_webdataset/resolve/main/en__*__{{000000..000006}}.tar"
url = f"pipe:curl -s -L {url} -H 'Authorization:Bearer {hf_token}'"

cache_dir = Path('~/.cache/wds/tmp/').expanduser()
os.makedirs(cache_dir, exist_ok=True)

def get_sample(item):
    txt = item['json']['raw_text']
    audio = item['wav']

    return txt, audio

In [ ]:
dataset = wds.WebDataset(url, shardshuffle=None, cache_dir=cache_dir).decode().map(get_sample)

# Create DataLoader
dataloader = DataLoader(
    dataset.batched(256),
    batch_size=None,
    num_workers=4
)